In [1]:
%cd ..

C:\Users\kubak\Desktop\engineer-project\github\LanguageModule


C:\Users\kubak\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import cohere
import os
import json

# Łączenie się z api cohere

In [3]:
# Inicjalizacja klienta Cohere
# https://cohere.com/
# https://docs.cohere.com/reference/embed

with open('./api.txt', 'r', encoding='utf-8') as file:
    for line in file:
        if 'cohere:' in line:
            # Wyciągamy tekst po "cohere:"
            key_api = line.split('cohere:')[1].strip()
            
co = cohere.Client(key_api)

# Pobieranie dokumentów

In [4]:
documents_path =  f"./data/decoded_pdfs/"

In [5]:
# Lista do przechowywania zawartości książek
documents = []

# Iteracja przez wszystkie pliki w folderze
for i, filename in enumerate(os.listdir(documents_path)):
    if filename.endswith(".json"):  # Sprawdzenie, czy plik ma rozszerzenie .json
        file_path = os.path.join(documents_path, filename)
        
        # Otwórz plik JSON i załaduj dane
        with open(file_path, "r", encoding="utf-8") as file:
            book_content = json.load(file)
        
        # Połącz zawartość wszystkich stron w jedną zmienną tekstową
        full_text = " ".join(book_content.values())

        #print(i, full_text)
        # Dodaj pełny tekst książki do listy documents
        documents.append(full_text)

# RAG - łączenie dokumentów

In [6]:
# Funkcja do wyszukiwania dokumentów w zbiorze
def retrieve_relevant_documents(query, documents):
    response = co.rerank(
        model="rerank-english-v2.0",
        query=query,
        documents=documents,
        top_n=1  # Liczba dokumentów do zwrócenia
    )
    return response

# Funkcja do generowania odpowiedzi
def generate_response(query, context):
    response = co.generate(
        model="command",  # Model generowania Cohere
        prompt=f"Użytkownik pyta: {query}\n"
               f"Kontekst: {context}\n"
               f"Odpowiedz:",
        max_tokens=100,  # Maksymalna długość odpowiedzi
        temperature=0.7  # Stopień kreatywności odpowiedzi
    )
    return response.generations[0].text.strip()



In [7]:
# Przykład zapytania użytkownika
query = "Why do we need to squat low bar?"

# Krok 1: Znalezienie najbardziej adekwatnego dokumentu
retrieval_result = retrieve_relevant_documents(query, documents)
most_relevant_document = retrieval_result.results[0].document

# Krok 2: Wygenerowanie odpowiedzi w oparciu o znaleziony kontekst
response = generate_response(query, most_relevant_document)


In [8]:
# Wyświetlenie wyniku
print("Pytanie użytkownika:", query)
print("Najbardziej adekwatny dokument:", most_relevant_document)
print("Odpowiedź:", response)

Pytanie użytkownika: Why do we need to squat low bar?
Najbardziej adekwatny dokument: None
Odpowiedź: Squatting low bar is beneficial to strengthen the lower body and core muscles. The squat is a functional exercise that replicates getting up and sitting down, which is a daily movement. This variation of the squat also helps improve your posture and lower body alignment, as it focuses on the hamstring and posterior chain muscles rather than the quadriceps. 

Developing strength in the squat movement is important to improve leg and core strength and overall stability. This exercise also helps improve mobility if you consistently go shallow
